# Init

In [1]:
from utils import *
import vertexai
from vertexai.generative_models import GenerativeModel
from pprint import pprint as pp
import json
import time

In [2]:
vertexai.init(project='diplom-440610', location='us-central1')

In [3]:
gemini15pro = GenerativeModel("gemini-1.5-flash")

# Prompts

## All text prompt

### Format

In [4]:
all_text_format = '''{
    questions: [
        {
            question: str,
            textPart: str,
            answers: [
                {
                    answer: str,
                    isCorrect: boolean,
                }
            ]
        }
    ]
}'''

### Prompt

In [5]:
all_text_prompt = '''Your task is to read a part of the book carefully and create questions for the children who have read the book. 
You must create {} questions. Each question should have 4 answer choices with 1 correct answer.
Try to make the answer choices more similar to the text itself.
Avoid creating tricky questions.
The questions must be in this json format:
```
{}
```

The textPart field of the question should contain a part of the text from which you took the question

Here is a part of a book:
{}

The question must be in the same language as the book'''

## Splitted text prompt

### Format

In [6]:
split_text_format = '''{
    question: str,
    textPart: str,
    answers: [
        {
            answer: str,
            isCorrect: boolean,
        }
    ]
}'''

### Prompt

In [7]:
split_text_prompt = '''Your task is to read a part of the book carefully and create only ONE question for the children who have read this part. 
Question should have 4 answer choices with 1 correct answer.
Try to make the answer choices more similar to the text itself.
Avoid creating tricky questions.
The question must be in this json format:
```
{}
```

The textPart field of the question should contain a part of the text from which you took the question

Here is a part of a book:
{}

The question must be in the same language as the book
'''

## Prompt utils

### Get formatted all text prompt

In [8]:
def get_all_text_prompt(questions_amount: int, text_part: str):
    return all_text_prompt.format(questions_amount, all_text_format, text_part)

### Get formatted splitted text prompt

In [9]:
def get_split_text_prompt(text_part: str):
    return split_text_prompt.format(split_text_format, text_part)

# Generate answers

## All text

In [27]:
text = read_pdf('books/kobylianska-olha-iulianivna-Valse-melancolique-(melankholiynyy-vals)2457.pdf')
prompt = get_all_text_prompt(10, text)
response = gemini_generate_answer(gemini15pro, prompt)  

In [28]:
print(response.text)

{
"questions": [
{
"question": "Чому головна героїня не може слухати меланхолійної музики?",
"textPart": "Не можу слухати меланхолійної музики.",
"answers": [
{
"answer": "Вона не любить музику.",
"isCorrect": false
},
{
"answer": "Вона не розуміє музику.",
"isCorrect": false
},
{
"answer": "Вона не може опертися настроєві сумному, який вона відчуває.",
"isCorrect": true
},
{
"answer": "Вона вважає, що меланхолійна музика негарна.",
"isCorrect": false
}
]
},
{
"question": "Що головна героїня відчуває, коли слухає музику, яка спочатку звучить весело, а потім стає сумною?",
"textPart": "А вже найменше такої, що приваблює зразу душу ясними, до танцю визиваючими граціозними звуками, а відтак, зрікаючися їх незамітно, ллється лиш одною широкою струєю смутку!",
"answers": [
{
"answer": "Вона відчуває радість.",
"isCorrect": false
},
{
"answer": "Вона відчуває смуток.",
"isCorrect": true
},
{
"answer": "Вона відчуває здивування.",
"isCorrect": false
},
{
"answer": "Вона відчуває гнів.",
"isCo

In [26]:
print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "{\n\"questions\": [\n{\n\"question\": \"Чому головна героїня не може слухати меланхолійної музики?\",\n\"textPart\": \"Не можу слухати меланхолійної музики.\",\n\"answers\": [\n{\n\"answer\": \"Їй не подобається, як вона звучить.\",\n\"isCorrect\": false\n},\n{\n\"answer\": \"Вона занадто емоційна і легко піддається настрою.\",\n\"isCorrect\": true\n},\n{\n\"answer\": \"Вона вважає її занадто сумною.\",\n\"isCorrect\": false\n},\n{\n\"answer\": \"Вона не розуміє її.\",\n\"isCorrect\": false\n}\n]\n},\n{\n\"question\": \"Що головна героїня відчуває, коли слухає яскраву музику?\",\n\"textPart\": \"Зате, як пронесеться музика блиску, я подвійно живу.\",\n\"answers\": [\n{\n\"answer\": \"Вона відчуває себе щасливою.\",\n\"isCorrect\": true\n},\n{\n\"answer\": \"Вона відчуває себе сумною.\",\n\"isCorrect\": false\n},\n{\n\"answer\": \"Вона відчуває себе розчарованою.\",\n\"isCorrect\": false\n},\n{\n\"answer\": \"Вона відчу

In [ ]:
json_str = json.loads(response.text)
pp(json_str, indent=4)

## Splitted text

In [17]:
text = read_epub('books/vovchok-marko-vedmid-sestrychka-melasia1976.epub')
parts = split_text_into_parts(text, 10)
for part in parts:
    prompt = get_split_text_prompt(part)
    response = gemini_generate_answer(gemini15pro, prompt)  
    print(response.text)
    print("================================")
    time.sleep(30)

{
"question": "Хто є автором твору \"Ведмідь (Сестричка Мелася)\"?",
"textPart": "Ведмідь (Сестричка Мелася)\nМарко Вовчок",
"answers": [
{
"answer": "Марко Вовчок",
"isCorrect": true
},
{
"answer": "Падолист",
"isCorrect": false
},
{
"answer": "Сестричка Мелася",
"isCorrect": false
},
{
"answer": "Ведмідь",
"isCorrect": false
}
]
}

{
    "question": "Яка з цих речей НЕ була куплена Михайлом на ярмарку в Кролевці?",
    "textPart": "Тоді ж покупив він і чоботи свої з червоним закотом, і ту шапку чорну, що він її дуже прилюбляє й омалененько бере на себе, і що як візьме її, то дід сміється і каже: \"Гледи, Михайло, гледи! Рознесуть твою славну шапку лакомі бджоли на мед!\"",
    "answers": [
        {
            "answer": "Чоботи з червоним закотом",
            "isCorrect": false
        },
        {
            "answer": "Шапка чорна",
            "isCorrect": false
        },
        {
            "answer": "Сорочка біла",
            "isCorrect": true
        },
        {
        